In [96]:
import numpy as np
import sympy as sp

In [97]:
p = 65537
H_0 = [291, 17767, 35243, 52719, 65244, 47768, 568, 2]
N = [[65281, 4335, 289, 61170],
     [3823, 8992, 53012, 65248],
     [8447, 61748, 56545, 4335],
     [4369, 57090, 3823, 256]
     ]
X = [14, 341, 99, 10321, 100, 2323, 35, 16]
U = [4081, 256, 1, 61681]
U_ = [65521, 4352, 1, 0]

In [98]:
def yy(y):
    if y == 0:
        return y + 1
    else:
        return y

In [99]:
def Rec(x):
    y = []
    y.append(x[0] + yy(x[15]) * yy(x[14]) + yy(x[13]) + 1)
    y[0] = y[0] % p
    y.append(x[1] + yy(y[0])  * yy(x[15]) + yy(x[14]) + 2)
    y[1] = y[1] % p
    y.append(x[2] + yy(y[1])  * yy(y[0])  + yy(x[15]) + 4)
    y[2] = y[2] % p
    for i in range(3, 16):
        y.append(x[i] + yy(y[i-1]) * yy(y[i-2]) + yy(y[i-3]) + Zpow(2, i, p))
        y[i] = y[i] % p
    return y

In [100]:
def Zpow(a, p, m):
    result = 1
    while p > 2: # когда степень сократится до квадрата и меньше - завершаем
        if p % 2 == 0: # если степень кратна 2
            a = (a ** 2) % m
            p = p // 2 # целочисленное деление (на всякий)
        else:
            result = (result * a) % m
            p = p - 1
    a = (a ** p) % m
    result = (result * a) % m
    return result

In [101]:
def FT(x):
    y = []
    for i in range(0, 8):
        s = 0
        for j in range(0, 8):
            s += (Zpow(2, 4 * i * j, p) * x[j])
        y.append(s % p)
    return y

In [102]:
def mix(x, y):
    arr = []
    for i in range(len(x)):
        arr.append(x[i])
        arr.append(y[i])
    return arr

In [103]:
def FT2(x):
    ch = []
    nch = []
    for i in range(0, 15, 2):
        ch.append(x[i])
        nch.append(x[i + 1])
    ch = FT(ch)
    nch = FT(nch)
    return mix(ch, nch)

In [104]:
def A(x):
    return H_0 + x

def B(x):
    return Rec(A(x))

def C(x):
    return FT2(B(x))

def D(x):
    return Rec(C(x))

In [105]:
a = X[5]
b = X[6]
c = X[7]

In [106]:
def my_psi(Y, X_R):
    ch = []
    nch = []
    for i in range(len(Y)):
        if i % 2 == 0:
            ch.append(Y[i])
        else:
            nch.append(Y[i])
    result = np.array(mix(np.dot(N, ch), np.dot(N, nch))) + np.array(list([0, 0, 0, 0]) + list(X_R))
    return result

In [107]:
def rec_minus(y):
    x = []
    for i in range(8, 16):
        x.append(y[i] - yy(y[i-1]) * yy(y[i-2]) - yy(y[i-3]) - Zpow(2, i, p))
    return x

In [108]:
    Y = B(X)[0:8]

    X_rand = np.random.randint(0, 65538, 4)
    X_rand = X_rand.astype(np.int64) 
    arg = [0, 0, 0, 0, 0, 0, 0, 0] + mix([0, 0, 0, 0], X_rand)
    R = FT2(arg)[11:16]
    z_wave = my_psi(Y, X_rand)
    z_wave = np.array(z_wave, dtype=np.int64)
    z_wave = [x % p for x in z_wave]
    a_ = a + z_wave[4] * z_wave[3] + z_wave[2] + Zpow(2, 13, p) - z_wave[5]
    b_ = b + z_wave[5] * z_wave[4] + z_wave[3] + Zpow(2, 14, p) - z_wave[6]
    c_ = c + z_wave[6] * z_wave[5] + z_wave[4] + Zpow(2, 15, p) - z_wave[7]
    a_ = a_ % p
    b_ = b_ % p
    c_ = c_ % p
    mu = np.random.rand()
    first = np.array([[z_wave[3] + 256 * mu + 256, z_wave[3] + 256 * mu + 4352], 
                        [z_wave[5] + mu - 61681, z_wave[5] + mu]])
    second = np.array([- a_ + (1 - 256 * z_wave[4]) * mu, 
                       - b_ - (256 + z_wave[4]) * mu])
    answer = np.linalg.solve(first, second)
    Z = np.array(z_wave) + answer[0] * np.array(list(U)+list([0,0,0,0])) + answer[1] *  np.array(list(U_)+list([0,0,0,0])) + mu * np.array(list([0,0,0,0])+list(U))
    Z = [x % p for x in Z]
    res = rec_minus(Y + Z)
    res = [x % p for x in res]
    print(C(res)[11:16])

[22995.812415122986, 46739.2784576416, 42396.00490760803, 44370.28460788727, 32144.58408164978]
